# REopt

- API structure: https://developer.nrel.gov/api/reopt/stable/help?API_KEY=DEMO_KEY
- Models.py: https://github.com/NREL/REopt_API/blob/master/reoptjl/models.py

In [1]:
import os, sys, json
import shutil
import time
import yaml
import schedule
import pandas as pd
from src.post_and_poll import get_api_results
from utils import import_df, import_json, export_json, plotly_stacked, parse_dispatch_series, setup

# to silence warnigns
# InsecureRequestWarning: Unverified HTTPS request is being made to host 'developer.nrel.gov'. Adding certificate verification is strongly advised.
import urllib3
urllib3.disable_warnings()

pd.options.plotting.backend='plotly'

API_KEY = '0bgdMrMpcFEfzIuYLRFpEfPeUqabOQKo5RagyRPV' 

In [2]:
def create_post(cfg,array,solar_kw,batt_kw,batt_h,solar_kw_range=False,batt_kw_range=False,batt_h_range=False):
    post = cfg.post
    
    post['PV'].update({ "min_kw":solar_kw,
                        "max_kw":solar_kw,            
                        "production_factor_series":import_df(cfg.solar_file,array,
                                                             resamp=cfg.solar_resamp,
                                                             scale=cfg.solar_scaler)})
    if solar_kw_range:
        post['PV'].update({ "min_kw":0})
    
    post['ElectricLoad'].update({'loads_kw':import_df(cfg.load_file,cfg.load_col,
                                                      resamp=cfg.load_resamp)})
            
    post['ElectricStorage'].update({
            "min_kw":batt_kw,               "max_kw":batt_kw,
            "min_kwh":batt_kw*batt_h,       "max_kwh":batt_kw*batt_h,})
    if batt_kw_range:
        post['ElectricStorage'].update({"min_kw":0})
    if batt_h_range:
        post['ElectricStorage'].update({"min_kwh":0})    

    post["ElectricTariff"].update({
        "urdb_response":import_json(cfg.tariff_file),
        'wholesale_rate':[cfg.energy_price_sell_constant]*(8760*post['Settings']['time_steps_per_hour']),
        #'wholesale_rate':import_df('data/PG&E NBT EEC Values 2024 Vintage.csv','Price'),
    })
    
    export_json(post,cfg.outdir)
    
    return post

def run_reopt(post, print_results=False):

    outputs_file_name = "results_file"
    root_url = "https://developer.nrel.gov/api/reopt/stable" # /stable == /v3 
    
    try:
        api_response = get_api_results(post=post, 
                                    API_KEY=API_KEY, 
                                    api_url=root_url, 
                                    results_file= f'outputs/{outputs_file_name}.json', 
                                    run_id=None)
    except:
        print('API request failed')
        
    if api_response is not None:
        cost =      api_response["outputs"]["ElectricTariff"]["year_one_energy_cost_before_tax"]
        revenue =   api_response["outputs"]["ElectricTariff"]["year_one_export_benefit_before_tax"]
        netcost =   cost - revenue
        npv =       api_response['outputs']['Financial']['npv']
        spp =       api_response['outputs']['Financial']['simple_payback_years']

        if print_results:    
            print('Status = ',              api_response["status"])
            print("Energy cost ($) = ",     cost)
            print('Energy revenue ($) = ',  revenue)
            print('Net cost ($) = ',        netcost)
            print('NPV ($) and Payback Period (y) = ',        npv, spp)
            print('PV Size (kW) = ',        api_response["outputs"]["PV"]["size_kw"])
            if "ElectricStorage" in api_response["outputs"].keys():
                print('Storage Size (kW-kwh) = ',api_response["outputs"]["ElectricStorage"]["size_kw"],'-',api_response["outputs"]["ElectricStorage"]["size_kwh"])
    else:
        print('API request failed')
        cost,netcost = pd.NA,pd.NA
        
    return api_response, cost, netcost
        
def compare_to_s20(cfg,solar_kw:int,batt_kw:int,batt_h:int,print_results=False):
    cost_red,cost_red_pct,netcost_red = [], [], []
    post = create_post(cfg,'0w90',solar_kw,batt_kw,batt_h)
    _, cost_base, netcost_base = run_reopt(post,print_results)
    for array in cfg.arrays:
        try:
            post = create_post(cfg,f'{array}w90',solar_kw,batt_kw,batt_h)
            _, cost, netcost = run_reopt(post,print_results)
            cost_red.append(        cost_base - cost)
            cost_red_pct.append(    1 - cost/abs(cost_base))
            netcost_red.append(     netcost_base - netcost)
            if print_results:
                print(f'\nCost Reduction {array} ($) = {cost_base-cost:.2f} ({100*(1-cost/abs(cost_base)):.1f}%)')
                print(f'NetCost Reduction {array} ($) = {netcost_base-netcost:.2f} ({100*(1-netcost/abs(netcost_base)):.1f}%)\n')
        except:
            cost_red.append(        pd.NA)
            cost_red_pct.append(    pd.NA)
            netcost_red.append(     pd.NA)
    return cost_base, cost_red, cost_red_pct, netcost_red

# Setup

In [3]:
cfg = setup('reopt.yaml')

# Single run

In [4]:
post = create_post(cfg,'0w90',200,25,2)
api_response, cost, netcost = run_reopt(post,print_results=True)
print(cost, netcost)

Status =  optimal
Energy cost ($) =  4348.73
Energy revenue ($) =  7722.0
Net cost ($) =  -3373.2700000000004
NPV ($) and Payback Period (y) =  -181360.43 0.0
PV Size (kW) =  200.0
Storage Size (kW-kwh) =  25.0 - 50.0
4348.73 -3373.2700000000004


# Grid search

In [ ]:
cols = ['SolarKW','BattKW','BattH','BaseCost']
df = pd.DataFrame([],columns=cols   + [f'Reduc_{x}w90' for x in cfg.arrays]\
                                    + [f'Reduc%_{x}w90' for x in cfg.arrays]\
                                    + [f'NetReduc_{x}w90' for x in cfg.arrays])

#best_cost_red_pct = 0
#best_resp = None

if cfg.test:
    cfg.solar_kws   = [cfg.solar_kws[0]]
    cfg.batt_kws    = [cfg.batt_kws[0]]
    cfg.batt_hs     = [cfg.batt_hs[0]]

for solar_kw in cfg.solar_kws:
    for batt_kw in cfg.batt_kws:
        for batt_h in cfg.batt_hs:
            basecost, cost_red, cost_red_pct, netcost_red = compare_to_s20(cfg,solar_kw,batt_kw,batt_h,print_results=False)
            df.loc[len(df)] = [solar_kw,batt_kw,batt_h,basecost]+cost_red+cost_red_pct+netcost_red
            df.to_csv(cfg.outdir+'results.csv',index=False)
            print(df.iloc[:,:7])
            
            # for x,r,n in zip(cost_red_pct,resp,arrays):
            #     if x > best_cost_red_pct:
            #         best_cost_red_pct = x
            #         best_resp = r
            #         df.iloc[-1,:].to_csv(f'outputs/best_result_{n}.csv')
            
df

# Plot

In [ ]:
# dispatch = parse_dispatch_series(api_response)
# f = plotly_stacked(dispatch,solar='pv',load='load',batt='batt',utility='grid',soc='soc',theme='plotly_dark')

# Schedule

In [2]:
import sys
import json

def urdb_builder(d,h,pe1,pe2,pd2):

    class dotdict(dict):
        __getattr__ = dict.__getitem__
        __setattr__ = dict.__setitem__
        __delattr__ = dict.__delitem__
        
        
    # d = 2
    # h = 6
    # pe1 = 0.10
    # pe2 = 0.20    
    # pd2 = 20

    # if len(sys.argv) > 1:
    #     for i in range(1,len(sys.argv)):
    #         if sys.argv[i] == '-d':
    #             d = int(sys.argv[i+1])
    #             print('h =',h)
    #         if sys.argv[i] == '-h':
    #             h = int(sys.argv[i+1])
    #             print('hbegin = ',h)
    #         if sys.argv[i] == '-pe':
    #             pe1 = float(sys.argv[i+1])
    #             pe2 = float(sys.argv[i+2])
    #             print('pe = ',pe1,pe2)
    #         if sys.argv[i] == '-pd':
    #             pd2 = float(sys.argv[i+1])
    #             print('pd = ',pd2)
                
    t = dotdict()

    def add_level(t):
        return [0]*t[0] + [1]*(t[1]-t[0]) + [0]*(24-t[1])

    def make_row(pairs):
        row = [0]*24
        for pair in pairs:
            row = [x+y for x,y in zip(row,add_level(pair))]
        return row

    name = f'd{d}_h{h}_pe{pe1}-{pe2}_pd{pd2}'

    # week days
    t.energyweekdayschedule = [make_row([(0,24),(h,h+d)]) for x in range(12)]
    t.demandweekdayschedule = [make_row([(0,24),(h,h+d)]) for x in range(12)]

    # week ends
    t.energyweekendschedule = [make_row([(0,24)]) for x in range(12)]
    t.demandweekendschedule = [make_row([(0,24)]) for x in range(12)]

    t.energyratestructure = [
        [{ "rate": 0, "unit": "kWh" }],
        [{ "rate": pe1, "unit": "kWh" }],
        [{ "rate": pe2, "unit": "kWh" }]]

    t.demandratestructure = [
        [{ "rate": 0 }],
        [{ "rate": 0 }],
        [{ "rate": pd2 }]]


    with open(f'data/tariff/{name}.json', 'w') as fp:
        json.dump(t, fp)
        
    return name

In [3]:
def job(d,h,pe1,pe2,pd2):
    try:
        name = urdb_builder(d,h,pe1,pe2,pd2)
        with open('syn_tar.yaml', 'r') as f:
            d=yaml.safe_load(f)
            
            d['tariff_file'] = f'tariff/{name}.json'
            d['note'] = name
            
        with open('syn_tar.yaml', 'w') as f:
            yaml.dump(d, f)
            
        cfg = setup('syn_tar.yaml')
        
        cols = ['SolarKW','BattKW','BattH','BaseCost']
        df = pd.DataFrame([],columns=cols   + [f'Reduc_{x}w90' for x in cfg.arrays]\
                                            + [f'Reduc%_{x}w90' for x in cfg.arrays]\
                                            + [f'NetReduc_{x}w90' for x in cfg.arrays])
        if cfg.test:
            cfg.solar_kws   = [cfg.solar_kws[0]]
            cfg.batt_kws    = [cfg.batt_kws[0]]
            cfg.batt_hs     = [cfg.batt_hs[0]]

        for solar_kw in cfg.solar_kws:
            for batt_kw in cfg.batt_kws:
                for batt_h in cfg.batt_hs:
                    basecost, cost_red, cost_red_pct, netcost_red = compare_to_s20(cfg,solar_kw,batt_kw,batt_h,print_results=False)
                    df.loc[len(df)] = [solar_kw,batt_kw,batt_h,basecost]+cost_red+cost_red_pct+netcost_red
                    df.to_csv(cfg.outdir+'results.csv',index=False)
                    print(df.iloc[:,:7])
        print('job succeess')
    except:
        print('job failed')

In [5]:
# schedule.every().day.at('00:33').do(job,tariff_name='h4_18-22')

# schedule.every().day.at('01:03').do(job,tariff_name='h2_6-8')
# schedule.every().day.at('01:33').do(job,tariff_name='h2_8-10')
# schedule.every().day.at('02:03').do(job,tariff_name='h2_10-12')
# schedule.every().day.at('02:33').do(job,tariff_name='h2_12-14')
# schedule.every().day.at('03:03').do(job,tariff_name='h2_14-16')
# schedule.every().day.at('03:33').do(job,tariff_name='h2_16-18')
# schedule.every().day.at('04:03').do(job,tariff_name='h2_17-19')
# schedule.every().day.at('04:33').do(job,tariff_name='h2_18-20')
# schedule.every().day.at('05:03').do(job,tariff_name='h2_19-21')
# schedule.every().day.at('05:33').do(job,tariff_name='h2_20-22')

# schedule.every().day.at('06:03').do(job,tariff_name='h3_6-9')
# schedule.every().day.at('06:33').do(job,tariff_name='h3_8-11')
# schedule.every().day.at('07:03').do(job,tariff_name='h3_10-13')
# schedule.every().day.at('07:33').do(job,tariff_name='h3_12-15')
# schedule.every().day.at('08:03').do(job,tariff_name='h3_14-17')
# schedule.every().day.at('08:33').do(job,tariff_name='h3_16-19')
# schedule.every().day.at('09:03').do(job,tariff_name='h3_17-20')
# schedule.every().day.at('09:33').do(job,tariff_name='h3_18-21')
# schedule.every().day.at('10:03').do(job,tariff_name='h3_19-22')

while True:
    schedule.run_pending()
    time.sleep(1)

KeyboardInterrupt: 

In [14]:
runs = []

for pe1 in [0.05,0.10,0.15,0.20]:
    for deltap in [0.05,0.10,0.15,0.20]:
        pe2 = pe1+deltap
        for pd2 in [10,20,30,40]:
            runs.append((pe1,pe2,pd2))

H,M = 0,15
for run in runs:
    pe1,pe2,pd2 = run
    M += 15
    if M >= 60:
        H += 1
        M = M - 60
    time = str(H).zfill(2)+':'+str(M).zfill(2)
    schedule.every().day.at(time).do(job,d=8,h=14,pe1=pe1,pe2=pe2,pd2=pd2)
    
while True:
    schedule.run_pending()
    time.sleep(1)

00:30 0.05 0.1 10
00:45 0.05 0.1 20
01:00 0.05 0.1 30
01:15 0.05 0.1 40
01:30 0.05 0.15000000000000002 10
01:45 0.05 0.15000000000000002 20
02:00 0.05 0.15000000000000002 30
02:15 0.05 0.15000000000000002 40
02:30 0.05 0.2 10
02:45 0.05 0.2 20
03:00 0.05 0.2 30
03:15 0.05 0.2 40
03:30 0.05 0.25 10
03:45 0.05 0.25 20
04:00 0.05 0.25 30
04:15 0.05 0.25 40
04:30 0.1 0.15000000000000002 10
04:45 0.1 0.15000000000000002 20
05:00 0.1 0.15000000000000002 30
05:15 0.1 0.15000000000000002 40
05:30 0.1 0.2 10
05:45 0.1 0.2 20
06:00 0.1 0.2 30
06:15 0.1 0.2 40
06:30 0.1 0.25 10
06:45 0.1 0.25 20
07:00 0.1 0.25 30
07:15 0.1 0.25 40
07:30 0.1 0.30000000000000004 10
07:45 0.1 0.30000000000000004 20
08:00 0.1 0.30000000000000004 30
08:15 0.1 0.30000000000000004 40
08:30 0.15 0.2 10
08:45 0.15 0.2 20
09:00 0.15 0.2 30
09:15 0.15 0.2 40
09:30 0.15 0.25 10
09:45 0.15 0.25 20
10:00 0.15 0.25 30
10:15 0.15 0.25 40
10:30 0.15 0.3 10
10:45 0.15 0.3 20
11:00 0.15 0.3 30
11:15 0.15 0.3 40
11:30 0.15 0.35 10
1